# **Building neural net work**

In [2]:
#Data Manipulation
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from scipy.stats import zscore, loguniform, randint
import math

#Data visualization
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
colors = ['#c1121f','#669bbc', '#f4d35e', '#e9724c', '#ffc857']
plt.style.use('seaborn-white')
plt.rc('figure', figsize=(12,8))
plt.rc('font', size=18)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

#Preprocessing
from sklearn.model_selection import train_test_split
import sklearn.utils
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_array, check_is_fitted
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

#Models
from imblearn.over_sampling import RandomOverSampler
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

#Metrics and Tuning
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer
from yellowbrick.regressor import PredictionError, ResidualsPlot

#General
import os
from google.colab import files
import warnings
warnings.filterwarnings(action='ignore')

<ipython-input-2-7806d29d5089>:14: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')


In [3]:
df = pd.read_csv("/content/data_after_processing_all_100_factors.csv")
df.head()

,Country,People using safely managed drinking water services % of population 2017,People using safely managed sanitation services % of population 2017,Access to electricity % of population 2017,Renewable energy consumption % of total final energy consumption 2015_x,Expenditures for R&D % of GDP 2015,Urban population living in slums % of urban population 2014,Ambient PM2.5 air pollution mean annual exposure micrograms per cubic meter 2016,Adjusted net savings % of GNI 2017,Carbon dioxide emissions per capita metric tons 2014_x,Nationally protected terrestrial and marine areas % of total territorial area 2018,"Intentional homicides Combined source estimates per 100,000 people 2015",Internet use Individuals using the Internet % of population 2017,Rural population % of total 2000,Rural population % of total 2018,Rural population growth (annual %) 2018,Land area sq. km thousands 2016,Land use Forest area % of land area 2000,Land use Forest area % of land area 2016,Land use Permanent cropland % of land area 2000,Land use Permanent cropland % of land area 2016,Land use Arable land % of land area 2000,Land use Arable land % of land area 2016,Arable land hectares per person 2000,Arable land hectares per person 2016,Access to electricity % of population 2000,Access to electricity % of population 2016,Access to electricity % of urban population 2016,Access to electricity % of rural population 2016,Access to clean fuels and technologies for cooking % of population 2000,Access to clean fuels and technologies for cooking % of population 2016,Renewable energy consumption % of total final energy consumption 2000,Renewable energy consumption % of total final energy consumption 2015_y,Renewable electricity output % of total electricity output 2000,Renewable electricity output % of total electricity output 2015,Agricultural land % of land area 2000-02,Agricultural land % of land area 2014-16,Agricultural land % irrigated 2014-16,Average annual precipitation millimeters 2014,Land under cereal production hectares thousands 2000-02,Land under cereal production hectares thousands 2014-16,Fertilizer consumption % of fertilizer production 2014-16,Fertilizer consumption kilograms per hectare of arable land 2014-16,Agricultural employment % of total employment 2000-02,Agricultural employment % of total employment 2014-16,Agricultural machinery tractors per 100 sq. km of arable land 2000,Agricultural machinery tractors per 100 sq. km of arable land 2009,Exposure to impact Land area where elevation is below 5 meters % of land area 2010,Exposure to impact Population living in areas where elevation is below 5 meters % of total population 2010,"Exposure to impact Population affected by droughts, floods, and extreme temperatures average annual; % of total population 2009",Exposure to impact Urban land area where elevation is below 5 meters % of urban land area 2010,Exposure to impact Rural population living in areas where elevation is below 5 meters % of rural population 2010,Exposure to impact Rural land area where elevation is below 5 meters % of rural land area 2010,Exposure to impact Urban population living in areas where elevation is below 5 meters % of urban population 2010,"Resilience Disaster risk reduction progress score 1, worst to 5,best 2011",Net energy imports % of energy use 1990,Net energy imports % of energy use 2015,GDP per unit of energy use 2011 PPP $ per kilogram of oil equivalent 1990,GDP per unit of energy use 2011 PPP $ per kilogram of oil equivalent 2015,Carbon dioxide emissions Total thousand metric tons 1990,Carbon dioxide emissions Total thousand metric tons 2014,Carbon dioxide emissions Carbon intensity kilograms per kilograms of oil equivalent energy use 1990,Carbon dioxide emissions Carbon intensity kilograms per kilograms of oil equivalent energy use 2014,Carbon dioxide emissions per capita metric tons 1990,Carbon dioxide emissions per capita metric tons 2014_y,Carbon dioxide emissions kilograms per 2011 PPP $ of GDP 1990,Carbon

In [4]:
print('------ Dataset ------')
print(f'Number of rows: {df.shape[0]}')
print(f'Number of columns: {df.shape[1]}')

------ Dataset ------
Number of rows: 214
Number of columns: 101


In [5]:
df.set_index("Country", inplace = True)

In [6]:
df.columns

Index(['People using safely managed drinking water services % of population 2017',
       'People using safely managed sanitation services % of population 2017',
       'Access to electricity % of population 2017',
       'Renewable energy consumption % of total final energy consumption 2015_x',
       'Expenditures for R&D % of GDP 2015',
       'Urban population living in slums % of urban population 2014',
       'Ambient PM2.5 air pollution mean annual exposure micrograms per cubic meter 2016',
       'Adjusted net savings % of GNI 2017',
       'Carbon dioxide emissions per capita metric tons 2014_x',
       'Nationally protected terrestrial and marine areas % of total territorial area 2018',
       'Intentional homicides Combined source estimates per 100,000 people 2015',
       'Internet use Individuals using the Internet % of population 2017',
       'Rural population % of total 2000', 'Rural population % of total 2018',
       'Rural population growth (annual %)  2018',
       

In [7]:
# defining x and y and chose target as out put instead of Tuberculosis per 100,000
X1= df.drop(columns=['Incidence of tuberculosis per 100,000 people 2018','output']).values
y1 = df['Incidence of tuberculosis per 100,000 people 2018'].values

In [8]:
# split the data
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.25, random_state=0)

In [9]:
# scaling the data
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X1_train)

# Scale the data
X1_train_scaled = X_scaler.transform(X1_train)
X1_test_scaled = X_scaler.transform(X1_test)

X1_test_scaled.shape

(54, 98)

In [14]:
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=11, activation="relu", input_dim=98))
nn_model.add(tf.keras.layers.Dense(units=22, activation="softmax"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["mse"])

nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 11)                1089      
                                                                 
 dense_7 (Dense)             (None, 22)                264       
                                                                 
 dense_8 (Dense)             (None, 1)                 23        
                                                                 
Total params: 1,376
Trainable params: 1,376
Non-trainable params: 0
_________________________________________________________________


In [15]:
fit_model = nn_model.fit(X1_train_scaled, y1_train, epochs=200)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X1_test_scaled,y1_test,verbose=2)


Epoch 1/200
5/5 [==============================] - 1s 5ms/step - loss: 285.7346 - accuracy: 0.0250
Epoch 2/200
5/5 [==============================] - 0s 5ms/step - loss: 237.2340 - accuracy: 0.0250
Epoch 3/200
5/5 [==============================] - 0s 5ms/step - loss: 202.4224 - accuracy: 0.0250
Epoch 4/200
5/5 [==============================] - 0s 5ms/step - loss: 178.9189 - accuracy: 0.0250
Epoch 5/200
5/5 [==============================] - 0s 5ms/step - loss: 156.9407 - accuracy: 0.0250
Epoch 6/200
5/5 [==============================] - 0s 6ms/step - loss: 138.2791 - accuracy: 0.0250
Epoch 7/200
5/5 [==============================] - 0s 5ms/step - loss: 121.5417 - accuracy: 0.0250
Epoch 8/200
5/5 [==============================] - 0s 4ms/step - loss: 105.5259 - accuracy: 0.0250
Epoch 9/200
5/5 [==============================] - 0s 4ms/step - loss: 91.8093 - accuracy: 0.0250
Epoch 10/200
5/5 [==============================] - 0s 5ms/step - loss: 80.7101 - accuracy: 0.0250
Epoch 11/20

In [17]:
model_loss, model_accuracy = nn_model.evaluate(X1_test_scaled,y1_test,verbose=2)

2/2 - 0s - loss: -1.2356e+03 - accuracy: 0.0000e+00 - 25ms/epoch - 12ms/step


In [41]:
model.save('nn_model.h5')

In [16]:
! pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.6 MB/s eta 0:00:00


In [23]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh', 'softmax'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=31,
        step=5), activation=activation, input_dim=98))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=50,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["mse"])

    return nn_model

In [24]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective=kt.Objective("val_mse", direction='min'),
    max_epochs=100,
    hyperband_iterations=2)

In [25]:
# Run the kerastuner search for best hyperparameters
tuner.search(X1_train_scaled,y1_train,epochs=200,validation_data=(X1_test_scaled,y1_test))

Trial 500 Complete [00h 00m 12s]
val_mse: 18062.9375

Best val_mse So Far: 18062.9375
Total elapsed time: 00h 25m 41s


In [26]:
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 16, 'num_layers': 4, 'units_0': 25, 'units_1': 25, 'units_2': 37, 'units_3': 29, 'units_4': 39, 'tuner/epochs': 12, 'tuner/initial_epoch': 4, 'tuner/bracket': 4, 'tuner/round': 2, 'tuner/trial_id': '0104'}
{'activation': 'relu', 'first_units': 26, 'num_layers': 4, 'units_0': 9, 'units_1': 27, 'units_2': 41, 'units_3': 31, 'units_4': 41, 'tuner/epochs': 12, 'tuner/initial_epoch': 4, 'tuner/bracket': 4, 'tuner/round': 2, 'tuner/trial_id': '0107'}
{'activation': 'relu', 'first_units': 31, 'num_layers': 5, 'units_0': 5, 'units_1': 41, 'units_2': 9, 'units_3': 37, 'units_4': 47, 'tuner/epochs': 12, 'tuner/initial_epoch': 4, 'tuner/bracket': 4, 'tuner/round': 2, 'tuner/trial_id': '0120'}


In [29]:
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X1_test_scaled,y1_test,verbose=2)
    print(f"Loss: {model_loss}, mse: {model_accuracy}")

2/2 - 0s - loss: -8.3329e+03 - mse: 18062.9375 - 264ms/epoch - 132ms/step
Loss: -8332.8974609375, mse: 18062.9375


2/2 - 0s - loss: -3.4518e+03 - mse: 18062.9375 - 282ms/epoch - 141ms/step
Loss: -3451.78955078125, mse: 18062.9375
2/2 - 0s - loss: -4.3597e+03 - mse: 18062.9375 - 278ms/epoch - 139ms/step
Loss: -4359.7177734375, mse: 18062.9375


In [30]:
second_hyper = tuner.get_best_hyperparameters(2)[1]
second_hyper.values

{'activation': 'relu',
 'first_units': 26,
 'num_layers': 4,
 'units_0': 9,
 'units_1': 27,
 'units_2': 41,
 'units_3': 31,
 'units_4': 41,
 'tuner/epochs': 12,
 'tuner/initial_epoch': 4,
 'tuner/bracket': 4,
 'tuner/round': 2,
 'tuner/trial_id': '0107'}

In [35]:
# build a modle 1 with the best value from the tuner
nn_model1 = tf.keras.models.Sequential()
nn_model1.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=98))
nn_model1.add(tf.keras.layers.Dense(units=25, activation="relu"))
nn_model1.add(tf.keras.layers.Dense(units=25, activation="relu"))
nn_model1.add(tf.keras.layers.Dense(units=37, activation="relu"))
nn_model1.add(tf.keras.layers.Dense(units=29, activation="relu"))
nn_model1.add(tf.keras.layers.Dense(units=39, activation="relu"))
# Compile the Sequential model together and customize metrics
nn_model1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["mse"])

nn_model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 16)                1584      
                                                                 
 dense_14 (Dense)            (None, 25)                425       
                                                                 
 dense_15 (Dense)            (None, 25)                650       
                                                                 
 dense_16 (Dense)            (None, 37)                962       
                                                                 
 dense_17 (Dense)            (None, 29)                1102      
                                                                 
 dense_18 (Dense)            (None, 39)                1170      
                                                                 
Total params: 5,893
Trainable params: 5,893
Non-traina

In [36]:

fit_model = nn_model1.fit(X1_train_scaled, y1_train, epochs=20)



Epoch 1/20
5/5 [==============================] - 1s 4ms/step - loss: 935.1364 - mse: 32379.6250
Epoch 2/20
5/5 [==============================] - 0s 4ms/step - loss: 804.6161 - mse: 32374.2402
Epoch 3/20
5/5 [==============================] - 0s 4ms/step - loss: 715.4720 - mse: 32367.6602
Epoch 4/20
5/5 [==============================] - 0s 3ms/step - loss: 662.5809 - mse: 32360.2344
Epoch 5/20
5/5 [==============================] - 0s 4ms/step - loss: 613.1659 - mse: 32352.4375
Epoch 6/20
5/5 [==============================] - 0s 4ms/step - loss: 541.2101 - mse: 32341.4590
Epoch 7/20
5/5 [==============================] - 0s 4ms/step - loss: 450.1483 - mse: 32327.0820
Epoch 8/20
5/5 [==============================] - 0s 5ms/step - loss: 361.6762 - mse: 32312.4688
Epoch 9/20
5/5 [==============================] - 0s 4ms/step - loss: 242.2109 - mse: 32293.8633
Epoch 10/20
5/5 [==============================] - 0s 4ms/step - loss: 116.8451 - mse: 32271.5820
Epoch 11/20
5/5 [============

In [33]:
# # Compare the performance to the second-best model
# second_model = tuner.get_best_models(2)[1]
# model_loss, model_accuracy = second_model.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [39]:
# Evaluate the performance of model using the loss and predictive accuracy of the model on the test dataset.
model_loss, model_accuracy = nn_model1.evaluate(X1_test_scaled,y1_test,verbose=2)
print(f"Loss: {model_loss}, mse: {model_accuracy}")

2/2 - 0s - loss: -6.8734e+02 - mse: 17814.9219 - 44ms/epoch - 22ms/step
Loss: -687.3380126953125, mse: 17814.921875


In [38]:
model.save('nn_model1.h5')

In [42]:
model = tf.keras.models.load_model('/content/nn_model1.h5')

# Save model architecture to a Python (.py) file in Colab
model_architecture_code = model.to_json()
with open('/content/model_architecture.py', 'w') as f:
    f.write('model_architecture_code = \'\'\'')
    f.write(model_architecture_code)
    f.write('\'\'\'')

# Save model weights to a file
model.save_weights('/content/model_weights.h5')

In [43]:

# # Load the model architecture from the Python file in Colab
# from model_architecture import model_architecture_code
# loaded_model = tf.keras.models.model_from_json(model_architecture_code)

# # Load the model weights
# loaded_model.load_weights('/content/model_weights.h5')